In [1]:
import torch 
""" Pytorch 1.7.1 is required to install on the client to run this function.
    Other versions of Pytorch may work as well, but those versions are not tested
"""

def convert_yolov5_weights(python_model_path, cpp_model_path, imsize=(640, 480)):
    """ This function converts a yolov5 pretrained model in python into model weights
    that can be loaded by C++ based program, and used by the DLmodelzoo action set as well as dlpy.

    To run this function, it does not require the entire dlpy package.
    
    It does require the following components:
        1. Pytorch 1.7.1
        2  The yolov5 model source code by ultralytics, v5.0. 
           The requirement comes from the fact that the python model is saved in a format that depends on the source code directory.
           To download a version of the source code, you can use the following command:
                git clone https://github.com/ultralytics/yolov5.git
                cd yolov5
                git checkout v5.0
        
        3. Python model weights that are compatible to DLmodelzoo yolov5 models, i.e., the yolov5 models by ultralytics, v5.0.

            You can download the weights from the following links:

            https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5m.pt
            https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5s.pt
            https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5l.pt
            https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5x.pt

            Other versions of the model weights are not tested.
    """

    device = torch.device('cpu')  # or torch.device('cpu')
    ckpt = torch.load(python_model_path, map_location=device)

    # model = Ensemble()
    model = []
    model.append(ckpt['ema' if ckpt.get('ema') else 'model'].float().eval())
    model = model[-1]

    imsize_md=max(imsize[0], imsize[1])
    if device.type != 'cpu':
        model(torch.zeros(1, 3, imsize_md, imsize_md).to(device).type_as(next(model.parameters())))

    img = torch.rand([1, 3, imsize[0], imsize[1]]).to(device).float()

    pred = model(img, augment=False)[0]
    cpp_model = torch.jit.trace(model, img)
    cpp_model.save(cpp_model_path)


In [3]:
import sys


"""           
Variable 'yolov5_src_dir' contains the top level directory for the yolov5 source code by ultralytics, v5.0.
To download the source code, you can use the following command:
                git clone https://github.com/ultralytics/yolov5.git
                cd yolov5
                git checkout v5.0
"""
yolov5_src_dir = "./yolov5"         

"""
Variable 'python_model_path' contains the file name for the model weights.
You can download the weights from the following links:    
    https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5m.pt
    https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5s.pt
    https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5l.pt
    https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5x.pt
"""        
python_model_path = "./yolov5l.pt" 

# The file name in a local path to save the converted weights that can be loaded by the C++ program.
cpp_model_path = "./cpp_yolov5l.pt"  

sys.path.insert (0, yolov5_src_dir)


""" 
After successful completion of the following call 'convert_yolov5_weights', the weights is saved in a local directory 
indicated by the variable 'cpp_model_path'.
To use this weights in dlmodelzoo actionset, or dlpy, you can either
    1. Put this file in a directory belonging to a CAS library, and
       specify the path in the YAML　document for the training or scoring action, or 
    2. Upload it to the CAS table using dlpy function 'upload_model_from_client in the file dlpy/mzmode.py'. 
       You can see one example to do this in the function 'test_mzmodel_from_client' in the file 'dlpy/tests/test_mzmodel.py'
"""
convert_yolov5_weights(python_model_path, cpp_model_path)
